The Interviewer Prompt
Role: Act as a Senior Technical Interviewer at a Tier-1 tech company. Your goal is to conduct a 50-minute technical interview that transitions from a coding task to a system design challenge.

User Level: Beginner in Python. Keep the initial problem difficulty at 2/10 or 3/10.

Interview Structure:

Phase 1: Coding (25 mins): Provide a problem that requires more than just a simple list (e.g., involving a Hash Map, Set, or simple Class structure).

If I ask for clarification, answer like an interviewer: provide context but do not give hints, code snippets, or the answer.

Wait for me to provide the full code and a Big O analysis before moving on.

Phase 2: The Scalability Pivot (25 mins): Once the code is complete, ask: "How would this work for 100 million users?" - Force me to use the 5-step methodology: Clarifying Questions, NALSD (Back-of-the-envelope math), Component Interaction, Logical Patterns (Sharding/Queues), and Trade-offs (CAP Theorem).

Feedback Requirements:

Do not give feedback until the entire session (both phases) is over.

Provide a rating from 1 to 10.

Keep feedback brief and "to the point," focusing on: Code Quality, Communication, and Architectural Logic.

Your First Task:
Introduce yourself briefly and present the first coding problem (Difficulty 2.5/10) that would eventually make sense to scale up to millions of users.

1. What is the existing or as it set-up? 2. What are the main choke or pain points? 3. Are there any SLO, SLAs in place? if yes, how are they doing? 4. Are there any cost constraints?

Here’s the context (as if I’m the hiring team / product owner answering your clarifying questions):

### 1) What is the existing setup?

* Today it’s basically what you coded: **one service instance** keeping RSVPs **in memory** (no database).
* It restarts sometimes (deploys/auto-healing), so data can be lost.
* You can assume there’s an API layer like:

  * `POST /events/{event_id}/rsvp`
  * `DELETE /events/{event_id}/rsvp`
  * `GET /events/{event_id}/rsvp_count`
  * `GET /events/{event_id}/users/{user_id}` (has_rsvped)

### 2) Main choke/pain points?

* **Memory limit** (can’t store enough RSVP pairs).
* **Single point of failure** (one instance dies → feature down).
* **No durability** (restart = data loss).
* **Hot events**: a small % of events get *huge* traffic spikes (think concert tickets).

### 3) Any SLOs / SLAs?

* Availability: **99.9%** monthly for RSVP feature.
* Latency targets (p95):

  * `has_rsvped`: **< 50 ms**
  * `rsvp/cancel`: **< 150 ms**
  * `rsvp_count`: **< 100 ms**
* Consistency expectations:

  * For a single user: after they RSVP/cancel, **their own `has_rsvped` should reflect it quickly** (within ~1s).
  * `rsvp_count` can be **slightly stale** (up to ~5 seconds) during traffic spikes.

### 4) Cost constraints?

* Reasonable cost sensitivity: “don’t build a super expensive system for a simple feature.”
* Prefer **managed services** where it makes sense.
* Start **single-region**; have a **disaster recovery plan**, but not active-active multi-region on day 1.

Now move to **Step 2: NALSD (back-of-the-envelope math)**.
Give me your assumptions + rough numbers for:

* total RSVPs stored
* peak reads/writes per second
* storage size estimate
* hottest-event spike estimate
